In [1]:
from copy import copy
import shutil
import time
from MMC.lib.email import send_email
from MMC.lib.logger_settings import add_log_handlers
from MMC.lib.transfer import Movie, Transfer, check_files, remove_from_source, update_source
from MMC.lib.session import save_session, load_session_from_file, Session
from MMC.lib.scipion_workflow import scipion_template
from pathlib import Path
from MMC import settings
import traceback
import logging

In [5]:
session = '20230124_AR3_0120_2'
directory = 'staging'

In [6]:

file, session = load_session_from_file(session)
transfer_obj = Transfer(session.sourceDir, logdir=file.parent, filesPattern=session.filesPattern, gainReference=session.gainReference)
to_check = transfer_obj.total_files
location = settings.storageLocations[directory]
location.set_session_path(session.specific_path)
corrupted_directory = location.session_raw_path / 'corrupted'   

In [7]:
corrupted_files = transfer_obj.corrupted_files

In [8]:
corrupted_files

[Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_041_0.0_X+0Y+1-2.tif, status=['incomplete'] ,timestamp=1674579917.9252744, delete=False),
 Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_069_0.0_X+1Y+1-2.tif, status=['staging', 'corrupted'] ,timestamp=1674580197.731025, delete=False),
 Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_080_0.0_X-1Y+0-1.tif, status=['incomplete'] ,timestamp=1674580329.0767272, delete=False),
 Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_094_0.0_X+1Y+0-1.tif, status=['incomplete'] ,timestamp=1674580468.7520835, delete=False),
 Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_150_0.0_X-1Y+0-1.tif, status=['incomplete'] ,timestamp=1674581028.480935, delete=False),
 Movie(path=/mnt/krios_Raid_X/SerialEM_stacks/AR2_0120_2/20230124_AR3_0120_2_164_0.0_X+1Y+0-1.tif, status=['incomplete'] ,timestamp=1674581167.3047855, delet

In [ ]:
for file in corrupted_files:
    filename=file.path.name
    source_size = file.path.stat().st_size
    location_size = Path(location.session_path, filename).stat().st_size
    print(f'{filename}: {source_size}, {location_size}. {location_size == source_size}')